[⬅️ Volver al Índice](../README.md)

---


# 5.4 Lab: Mini ETL - Consolidación de Fuentes

- Objetivo: unir datos de ventas y suscripciones con FULL OUTER JOIN para ingresos totales diarios.
- Prerrequisitos: CTEs, COALESCE, FULL OUTER JOIN, GROUP BY.
- Entregables: consulta que combine ingresos de 2 fact tables por fecha.
- Errores comunes: COALESCE mal aplicado, fechas sin estandarizar, duplicar registros.

## ¿Por qué consolidar fuentes?

En entornos reales, los ingresos provienen de múltiples sistemas: ventas de productos, suscripciones recurrentes, servicios profesionales. Un reporte financiero completo requiere sumar todas estas fuentes.

**Para qué:**
- Visión unificada de ingresos totales
- Evitar doble contabilidad (duplicados)
- Identificar fechas sin actividad en alguna fuente

**Cómo:**
- FULL OUTER JOIN para incluir fechas presentes en cualquier fuente
- COALESCE para estandarizar claves (fechas con NULL)
- ISNULL/COALESCE para reemplazar NULLs por 0 en sumas
- CTEs para modularizar cada fuente antes de unir

## Conceptos Clave ETL

**Extract:** Obtener datos de cada fuente (fact_ventas, fact_suscripciones)
**Transform:** Agregar, limpiar, estandarizar fechas y monedas
**Load:** Insertar resultado en tabla consolidada o entregar a dashboard

### FULL OUTER JOIN vs INNER JOIN

- **INNER JOIN:** Solo fechas que existen en AMBAS fuentes (puede perder datos)
- **LEFT/RIGHT JOIN:** Prioriza una fuente, puede perder datos de la otra
- **FULL OUTER JOIN:** Incluye todas las fechas de ambas fuentes, rellenando con NULL donde no hay coincidencia

### COALESCE vs ISNULL

- **COALESCE(col1, col2, col3, ...):** Devuelve primer valor no NULL, estándar SQL
- **ISNULL(col, default):** Específico de T-SQL, solo 2 parámetros, ligeramente más rápido

Para sumas: `ISNULL(ingresos_ventas, 0)` asegura que NULL se trate como 0 en aritmética.

In [ ]:
-- Consolidación de ingresos diarios
WITH ventas AS (
  SELECT fecha, SUM(cantidad*p.precio_unitario*(1 - descuento_pct/100.0)) AS ingresos_ventas
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY fecha
), sus AS (
  SELECT fecha_inicio AS fecha, SUM(costo_mensual) AS ingresos_sus
  FROM dbo.fact_suscripciones
  GROUP BY fecha_inicio
)
SELECT COALESCE(v.fecha, s.fecha) AS fecha,
       ISNULL(ingresos_ventas,0) + ISNULL(ingresos_sus,0) AS ingresos_total
FROM ventas v FULL OUTER JOIN sus s ON v.fecha = s.fecha;

## Ejercicios Prácticos

🟢 **Básico:** Listar fechas donde hay ventas PERO NO hay suscripciones activas (usar LEFT JOIN con IS NULL)

🟠 **Intermedio:** Consolidar ingresos mensuales (GROUP BY YEAR(fecha), MONTH(fecha)) de ventas + suscripciones, mostrando % contribución de cada fuente

🔴 **Avanzado:** Agregar fact_inventario (costo de almacenamiento) a la consolidación, calculando margen neto = ingresos totales - costos inventario

⚙️ **Reto Senior:** Crear procedimiento almacenado que reciba @fecha_inicio y @fecha_fin, ejecute ETL incremental (solo fechas del rango), inserte resultado en tabla dbo.consolidado_ingresos con columnas: fecha, ingresos_ventas, ingresos_suscripciones, total, fecha_proceso (GETDATE())

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.